# PSK II. Makró blokk beadandó

### Instrukciók

  - Kövesd a lenti utasításokat
  - Ha végigértél a feladatokon, exportáld és mentsd le a notebookot HTML és ipynb formátumban (File $\to$ Download as: $\to$ HTML (.html) / Notebook (.ipynb))
  - Ha több részletben szeretnéd megoldani a feladatot, el tudod a böngésződbe a notebook aktuális állapotát a felhő alakú gombbal, a modelleket leíró fájlokat azonban külön kell letölteni
  - A beküldött megoldásnak hét fájlból kell állnia:
    - Válaszok a feltett kérdésekre doc/txt/pdf dokumentumban
    - A letöltött idősorok és a filterezett ciklikus komponensek csv/xlsx formátumban
    - Oszthatatlan munkával kibővített RBC modell viselkedési egyenleteinek levezetése (kézzel írva vagy gépelve)
    - Lefuttatott notebook HTML formátumban
    - Lefuttatott notebook ipynb formátumban
    - Módosított `rbc_baseline.yaml` fájl
    - Módosított `rbc_indivisible labor.yaml` fájl
  - Don't panic! Hosszúnak tűnik a notebook, de ez csak azt jelenti, hogy a munka nagy részét már megcsináltam neked :)

Jó munkát!

### Szabályok

  - Lehet (de nem kötelező) maximum három fős csoportokban dolgozni
  - A megoldásokat küldjétek el a martin.stancsics@gmail.com email címre a szakmai szabályzatban meghatározott határidőig
  - Bármilyen kérdést szintén bátran küldjetek a fenti email címre. Érdemes cc-zni a teljes évfolyamot, mert mást is érdekelhet a válasz

## 1. feladat: Adatok

  - Válassz egy szimpatikus országot (amihez rendelkezésre állnak hosszú idősorok), és töltsd le a következő adatokat (figyelj rá, hogy minden flow változó ugyanolyan frekvenciájú legyen):
    - Reál GDP
    - Reál fogyasztás
    - Reál beruházás
    - Reál tőkeállomány
    - Reál bérek
    - Ledolgozott órák száma
    - Munka részesedése a GDP-ből
  - Bontsd fel a fenti idősorok *logaritmusát* trendre és ciklikus komponensre Hodrick-Prescott (HP) filter segítségével
    - Ha lehetőséged van statisztikai szoftvert használni, szuper
    - Ha nem, a következő weboldal ugyanolyan jó: https://dge.repec.org/cgi-bin/hpfilter.cgi
    - A lambda paraméter értéke éves adatok esetén legyen 100, negyedéves adatok esetén pedig 1600

## 2. feladat: Egy kalibrált RBC modell

**Feladat**:
  - Számold ki a korábban letöltött, idősorok átlagait
  - Az RBC modellt a következő módon tudod kalibrálni:
    - $\alpha = 1 - \frac{wL}{Y} = 1 - $(munka részesedése a GDP-ből)
    - $\delta = I/K$
    - $\beta = \left( \alpha \frac{Y}{K} + 1 - \delta \right) ^ {-1}$
    - $\rho = \begin{cases} 0.95 \quad\text{negyedéves adatok esetén} \\ 0.8 \quad\text{éves adatok esetén} \end{cases}$
  - Helyettesítsd be a fenti értékeket az `rbc_baseline.yml` fájl megfelelő részére

Ha végeztél, futtasd le a következő cellákat és válaszolj a feltett kérdésekre!
  

In [ ]:
import dolo
model_baseline = dolo.yaml_import("rbc_baseline.yaml")
print(model_baseline)

### Steady state

In [ ]:
model_baseline.get_calibration()

### Szimuláció

Oldd meg a modellt és szimulálj 5000 időszaknyi adatot

In [ ]:
dr_baseline = dolo.approximate_controls(model_baseline)
sim_baseline = dolo.simulate(model_baseline, dr_baseline, N=1, T=5000).squeeze()
sim_df_baseline = sim_baseline \
    .to_dataframe(name="value") \
    .drop(columns="N") \
    .reset_index() \
    .pivot(index="T", columns="V") \
    ["value"]
sim_df_baseline.head(50).plot()

### Kiértékelés

Relatv volatilitások:

In [ ]:
from numpy import log
vars = ["y", "c", "i", "k", "l", "r", "w"]

In [ ]:
std_baseline = log(sim_df_baseline[vars]).std()
(std_baseline / std_baseline["y"]).to_frame(name="Relative standard deviation")

Korrelációk:

In [ ]:
log(sim_df_baseline[vars]).corr()

**Feladat:** Számold ki a letöltött idősorok hasonló statisztikáit
  - Volatilitás: a korábban lementett (log) ciklikus komponensek szórása
  - Relatív volatilitás: volatilitás osztva a GDP volatilitásával
  - Korrelációk: a korábban lementett (log) ciklikus komponensek közötti korrelációk

**Kérdések:**

  - Mennyire adja vissza jól a modell a megfigyelt korrelációkat?
  - Mi a helyzet a munka szórásával?
  - Mi a helyzet a bérek prociklikusságával?

## 3. feladat: Oszthatatlan munka

Ötlet: a háztartás nem döntheti el, hogy hány órát dolgozik, csak azt, hogy milyen valószínűséggel dolgozik nyolc órát. A háztartás várható hasznosságot maximalizál, ezért a hasznosságfüggvény gyakorlatilag lineáris lesz a munkára nézve:

$$ u(c_t, \ell_t) = \log(c_t) - \phi \ell_t $$

Más nem változik az eredeti modellhez képest.

**Feladat:** vezesd le az így módosított RBC modell viselkedési egyenleteit
  - Oldd meg a háztartás hasznosságmaximalizálási problémáját
  - Vezesd le a háztartás két (intratemporális és intertemporális) viselkedési egyenletét - mi változik?
  - Oldd meg a vállalat profitmaximalizálási feladatát (változik bármi?)
  - Használd ki, hogy egyensúlyban a munka -és tőkekereslet megegyezik a kínálatukkal. Ez alapján írd fel az RBC modell fő 5+1 egyenletét (mi változik?)
  - Módosítsd az `rbc_indivisible_labor.yaml` fájlt a megváltozott egyenleteknek megfelelően

Ha végeztél, a korábbiakhoz hasonlóan futtasd le a következő cellákat és válaszolj a feltett kérdésekre!

In [ ]:
import dolo
model_alternative = dolo.yaml_import("rbc_indivisible_labor.yaml")
print(model_alternative)

Tipp: ha jól vezetted le a viselkedési egyenleteket, a fenti egyenletek teljesülnek steady-stateben és a kettőspontok bal oldalán nullák vannak.

### Steady state

In [ ]:
model_alternative.get_calibration()

### Szimuláció

Oldd meg a modellt és szimulálj 1000 időszaknyi adatot

In [ ]:
dr_alternative = dolo.approximate_controls(model_alternative)
sim_alternative = dolo.simulate(model_alternative, dr_alternative, N=1, T=1000).squeeze()
sim_df_alternative = sim_alternative \
    .to_dataframe(name="value") \
    .drop(columns="N") \
    .reset_index() \
    .pivot(index="T", columns="V") \
    ["value"]
sim_df_alternative.head(50).plot()

### Kiértékelés

Relatív volatilitások

In [ ]:
from numpy import log
vars = ["y", "c", "i", "k", "l", "r", "w"]

In [ ]:
std_alternative = log(sim_df_alternative[vars].std())
(std_alternative / std_alternative["y"]).to_frame(name="Relative standard deviation")

Korrelációk:

In [ ]:
log(sim_df_alternative[vars]).corr()

**Kérdések:**

  - Mi változott?
  - Mi a helyzet a munka szórásával?
  - Mi a helyzet a bérek prociklikusságával?
  - Közelebb kerültünk a valós adatokhoz?
  